In [ ]:
#| default_exp target_transforms

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Target transforms
Transformations that can be applied to the target before fitting and restored after predicting.

In [ ]:
#| export
import abc
from typing import Callable, Tuple

try:
    from numba import njit
except ImportError:
    raise ImportError(
        "Please install numba. "
        "You can find detailed instructions at https://numba.pydata.org/numba-doc/latest/user/installing.html"
    )
import numpy as np

from utilsforecast.grouped_array import GroupedArray

In [ ]:
#| export
class BaseTargetTransform(abc.ABC):
    """Base class used for target transformations."""
    @abc.abstractmethod
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        raise NotImplementedError

    @abc.abstractmethod
    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        raise NotImplementedError

In [ ]:
#| exporti
@njit
def _standard_scaler_transform(
    data: np.ndarray,
    indptr: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    n_groups = len(indptr) - 1
    stats = np.empty((n_groups, 2))
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        mean = np.nanmean(data[sl])
        std = np.nanstd(data[sl])
        stats[i, :] = mean, std
        out[sl] = (data[sl] - mean) / std
    return out, stats


@njit
def _standard_scaler_inverse_transform(
    data: np.ndarray,
    indptr: np.ndarray,
    stats: np.ndarray,
) -> np.ndarray:
    n_groups = len(indptr) - 1
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        mean, std = stats[i]
        out[sl] = data[sl] * std + mean
    return out

In [ ]:
#| export
class LocalStandardScaler(BaseTargetTransform):
    """Standardizes each serie by subtracting its mean and dividing by its standard deviation."""
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        transformed, self.stats_ = _standard_scaler_transform(ga.data, ga.indptr)
        return transformed

    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        return _standard_scaler_inverse_transform(ga.data, ga.indptr, self.stats_)

In [ ]:
from utilsforecast.data import generate_series

In [ ]:
series = generate_series(10, min_length=50, max_length=100)

In [ ]:
sc = LocalStandardScaler()
data = series['y'].values
sizes = series.groupby('unique_id', observed=True).size().values
indptr = np.append(0, sizes.cumsum())
ga = GroupedArray(data, indptr)
transformed = sc.fit_transform(ga)
transformed_ga = GroupedArray(transformed, ga.indptr)
np.testing.assert_allclose(
    sc.inverse_transform(transformed_ga),
    data,
)

In [ ]:
#| export
class LocalBoxCox(BaseTargetTransform):
    """Finds optimum lambda for each serie and applies Box-Cox transformation."""
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        from scipy.stats import boxcox

        out = np.full(ga.data.shape, np.nan)
        self.lmbdas_ = np.empty(ga.n_groups)
        for i in range(ga.n_groups):
            sl = slice(ga.indptr[i], ga.indptr[i + 1])
            mask = ~np.isnan(ga.data[sl])
            transformed, self.lmbdas_[i] = boxcox(ga.data[sl][mask] + 1.0, lmbda=None)
            if np.isclose(transformed * self.lmbdas_[i], -1).any():
                # in this case we can't reliably invert the transformation
                # fallback to log
                self.lmbdas_[i] = 0.0
                transformed = np.log1p(ga.data[sl][mask])
            out[sl][mask] = transformed
        return out

    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        from scipy.special import inv_boxcox1p

        sizes = np.diff(ga.indptr)
        lmbdas = np.repeat(self.lmbdas_, sizes, axis=0)
        return inv_boxcox1p(ga.data, lmbdas)

In [ ]:
bc = LocalBoxCox()
transformed = bc.fit_transform(ga)
transformed_ga = GroupedArray(transformed, ga.indptr)
np.testing.assert_allclose(
    bc.inverse_transform(transformed_ga),
    data,
)

In [ ]:
#| exporti
@njit
def _minmax_scaler_transform(
    data: np.ndarray,
    indptr: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    n_groups = len(indptr) - 1
    stats = np.empty((n_groups, 2))
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        min_ = np.nanmin(data[sl])
        max_ = np.nanmax(data[sl])
        stats[i, :] = min_, max_
        out[sl] = (data[sl] - min_) / (max_ - min_)
    return out, stats


@njit
def _minmax_scaler_inverse_transform(
    data: np.ndarray,
    indptr: np.ndarray,
    stats: np.ndarray,
) -> np.ndarray:
    n_groups = len(indptr) - 1
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        min_, max_ = stats[i]
        out[sl] = data[sl] * (max_ - min_) + min_
    return out

In [ ]:
#| export
class LocalMinMaxScaler(BaseTargetTransform):
    """Scales each serie to be in the [0, 1] interval."""
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        transformed, self.stats_ = _minmax_scaler_transform(ga.data, ga.indptr)
        return transformed

    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        return _minmax_scaler_inverse_transform(ga.data, ga.indptr, self.stats_)

In [ ]:
sc = LocalMinMaxScaler()
transformed = sc.fit_transform(ga)
transformed_ga = GroupedArray(transformed, ga.indptr)
np.testing.assert_allclose(
    sc.inverse_transform(transformed_ga),
    data,
)

In [ ]:
#| exporti
@njit
def _robust_scaler_iqr_transform(
    data: np.ndarray,
    indptr: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    n_groups = len(indptr) - 1
    stats = np.empty((n_groups, 3))
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        q25, median, q75 = np.quantile(data[sl], (0.25, 0.5, 0.75))
        stats[i] = q25, median, q75
        out[sl] = (data[sl] - median) / (q75 - q25)
    return out, stats


@njit
def _robust_scaler_iqr_inverse_transform(
    data: np.ndarray,
    indptr: np.ndarray,
    stats: np.ndarray,
) -> np.ndarray:
    n_groups = len(indptr) - 1
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        q25, median, q75 = stats[i]
        out[sl] = data[sl] * (q75 - q25) + median
    return out


@njit
def _robust_scaler_mad_transform(
    data: np.ndarray,
    indptr: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    n_groups = len(indptr) - 1
    stats = np.empty((n_groups, 2))
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        median = np.median(data[sl])
        shifted = data[sl] - median
        mad = np.median(np.abs(shifted))
        stats[i] = median, mad
        out[sl] = shifted / mad
    return out, stats


@njit
def _robust_scaler_mad_inverse_transform(
    data: np.ndarray,
    indptr: np.ndarray,
    stats: np.ndarray,
) -> np.ndarray:
    n_groups = len(indptr) - 1
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        median, mad = stats[i]
        out[sl] = data[sl] * mad + median
    return out

In [ ]:
#| export
class LocalRobustScaler(BaseTargetTransform):
    """Scaler robust to outliers.

    Parameters
    ----------
    scale : str (default='iqr')
        Statistic to use for scaling. Can be either 'iqr' (Inter Quartile Range) or 'mad' (Median Asbolute Deviation)
    """
    def __init__(self, scale: str = 'iqr'):
        supported_scales = ('iqr', 'mad')
        if scale not in supported_scales:
            raise ValueError(f'scale must be one of {supported_scales}')
        self.scale = scale

    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        if self.scale == 'iqr':
            tfm_fn = _robust_scaler_iqr_transform
        else:
            tfm_fn = _robust_scaler_mad_transform
        transformed, self.stats_ = tfm_fn(ga.data, ga.indptr)
        return transformed

    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        if self.scale == 'iqr':
            inv_tfm_fn = _robust_scaler_iqr_inverse_transform
        else:
            inv_tfm_fn = _robust_scaler_mad_inverse_transform
        return inv_tfm_fn(ga.data, ga.indptr, self.stats_)

In [ ]:
for scale in ('iqr', 'mad'):
    sc = LocalRobustScaler(scale=scale)
    transformed = sc.fit_transform(ga)
    transformed_ga = GroupedArray(transformed, ga.indptr)
    np.testing.assert_allclose(
        sc.inverse_transform(transformed_ga),
        data,
    )

In [ ]:
#| export
class GlobalFuncTransformer(BaseTargetTransform):
    """Uses `func` and `inverse_func` for applying the same transformation to all series.

    Parameters
    ----------
    func : Callable
        Function that transforms the data.
    inverse_func : Callable
        Function that inverse transforms the data.
    """
    def __init__(self, func: Callable, inverse_func: Callable):
        self.func = func
        self.inverse_func = inverse_func
        
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        return self.func(ga.data)

    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        return self.inverse_func(ga.data)

In [ ]:
gt = GlobalFuncTransformer(np.log1p, np.expm1)
transformed = sc.fit_transform(ga)
transformed_ga = GroupedArray(transformed, ga.indptr)
np.testing.assert_allclose(
    sc.inverse_transform(transformed_ga),
    data,
)